In [177]:
# Set country to ADB country code of country
country = 'RUS'

In [178]:
# Import libraries and functions
import wbdata as wb                                       
import pandas as pd
from pandasgui import show
import numpy as np                                        
import datetime as dt                                     
import docx                                               
from docx.shared import Cm                                
from docx.enum.text import WD_ALIGN_PARAGRAPH             
from matplotlib import pyplot as plt 
from matplotlib import ticker
import random

In [179]:
# Create a dictionary of all indicators to be scraped from the World Bank API
all_indicators = {'NY.GDP.MKTP.KD.ZG': 'GDP growth (%)',
                  'NE.CON.PRVT.ZS': 'Private consumption (% of GDP)', 
                  'NE.CON.GOVT.ZS': 'Government expenditure (% of GDP)',
                  'NE.GDI.TOTL.ZS': 'Gross capital formation (% of GDP)', 
                  'NE.EXP.GNFS.ZS': 'Exports (% of GDP)',
                  'NE.IMP.GNFS.ZS': 'Imports (% of GDP)',
                  'NE.CON.PRVT.KD.ZG': 'Private consumption (annual growth, %)', 
                  'NE.CON.GOVT.KD.ZG': 'Government expenditure (annual growth, %)',
                  'NE.GDI.TOTL.KD.ZG': 'Gross capital formation (annual growth, %)',
                  'NE.EXP.GNFS.KD.ZG': 'Exports (annual growth, %)',
                  'NE.IMP.GNFS.KD.ZG': 'Imports (annual growth, %)',
                  'NE.EXP.GNFS.CD': 'Exports (current prices, USD)',
                  'NE.IMP.GNFS.CD': 'Imports (current prices, USD)',
                  'NV.AGR.TOTL.ZS': 'Agriculture (% of GDP)',
                  'NV.AGR.TOTL.KD.ZG': 'Agriculture (annual growth, %)',
                  'NV.IND.TOTL.ZS': 'Industry (including construction) (% of GDP)',
                  'NV.IND.TOTL.KD.ZG': 'Industry (including construction) (annual growth, %)',
                  'NV.SRV.TOTL.ZS': 'Services (% of GDP)',
                  'NV.SRV.TOTL.KD.ZG': 'Services (annual growth, %)',
                  'SL.UEM.TOTL.NE.ZS': 'Unemployment rate (%)',
                  'FP.CPI.TOTL.ZG': 'Inflation rate (%)',
                  'BN.CAB.XOKA.GD.ZS': 'Current account balance (% of GDP)',
                  'BN.GSR.GNFS.CD': 'Net trade in goods and services (current USD)',
                  'BN.GSR.MRCH.CD': 'Net trade in goods (current USD)',
                  'FI.RES.TOTL.CD': 'Total reserves (includes gold, current USD)',
                  'FI.RES.TOTL.MO': 'Total reserves in months of imports'}

# Set the time period
data_date = dt.datetime(dt.date.today().year - 6, 1, 1), dt.datetime(dt.date.today().year - 1, 1, 1)

# Create a dataframe of country codes and country names
country_codes = pd.DataFrame({'country_name': ['Russian Federation', 'Armenia', 'Azerbaijan', 
                                               'Georgia', 'Kazakhstan', 'Kyrgryz Republic', 
                                               'Tajikistan', 'Turkmenistan', 'Uzbekistan', 
                                               'Hong Kong, China', 'Mongolia', 
                                               'People\'s Republic of China', 'Republic of Korea',
                                               'Taipei,China', 'Afghanistian', 'Bangladesh', 
                                               'Bhutan', 'India', 'Maldives', 'Nepal', 'Pakistan', 
                                               'Sri Lanka', 'Brunei Darussalam', 'Cambodia', 
                                               'Indonesia', 'Lao People\'s Democratic Republic',
                                               'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 
                                               'Thailand', 'Timor-Leste', 'Viet Nam', 'Cook Islands',
                                               'Fiji', 'Kiribati', 'Marshall Islands', 'Nauru', 
                                               'Niue', 'Palau', 'Papua New Guinea', 'Samoa', 
                                               'Solomon Islands', 'Tonga', 'Tuvalu', 'Vanuatu'],
                              'comtrade_code': [643, 51, 31, 268, 398, 417, 762, 795, 860, 344, 496,
                                                156, 410, np.nan, 4, 50, 64, 699, 462, 524, 586, 144,
                                                96, 116, 360, 418, 458, 104, 608, 702, 764, 626, 704,
                                                184, 242, 296, 584, np.nan, np.nan, 585, 598, 882, 90,
                                                776, 798, 548]},
                               index=['RUS', 'ARM', 'AZE', 'GEO', 'KAZ', 'KGZ', 'TAJ', 'TKM', 'UZB',
                                      'HKG', 'MON', 'PRC', 'KOR', 'TAP', 'AFG', 'BAN', 'BHU', 'IND',
                                      'MLD', 'NEP', 'PAK', 'SRI', 'BRU', 'CAM', 'INO', 'LAO', 'MAL',
                                      'MYA', 'PHL', 'SIN', 'THA', 'TIM', 'VIE', 'COO', 'FIJ', 'KIR',
                                      'MHL', 'NAU', 'NIU', 'PAL', 'PNG', 'SAM', 'SOL', 'TON', 'TUV',
                                      'VAN'],
                             dtype='int64')

In [180]:
# Scrape data from the World Bank API
df_all = wb.get_dataframe(indicators=all_indicators, country=country, data_date=data_date, source=2,
                          convert_date=True)

# Sort data by year
df_all = df_all.sort_index()

# Generate share of net exports in GDP
net_exports_share = df_all['Exports (% of GDP)'] - df_all['Imports (% of GDP)']

# Insert 'net_exports_share' as the 6th column, labeled 'Net exports (% of GDP)'
df_all.insert(6, 'Net exports (% of GDP)', net_exports_share)

# Generate net exports annual growth
net_exports_growth = (df_all['Exports (current prices, USD)'] - df_all['Imports (current prices, USD)']).pct_change() * 100

# Insert 'net_exports_growth' as the 12th column, labeled 'Net exports (annual growth, %)'
df_all.insert(12, 'Net exports (annual growth, %)', net_exports_growth)

# Generate contributions to GDP growth using a loop
components = ['Private consumption', 'Government expenditure', 'Gross capital formation', 
              'Exports', 'Imports', 'Agriculture', 'Industry (including construction)', 'Services']
for item in components:
    df_all[f'{item} (contribution, pp)'] = df_all[f'{item} (% of GDP)'].shift(1) / 100 * df_all[f'{item} (annual growth, %)']

# Generate net export contribution to GDP growth
df_all['Net exports (contribution, pp)'] = df_all['Exports (contribution, pp)'] - df_all['Imports (contribution, pp)']

# Round to one decimal
df_all = df_all.round(1)

# Create dataframe for top three sources of growth on the demand side
demand_components = ['Private consumption', 'Government expenditure', 'Gross capital formation', 
                     'Net exports']
top_demand_contributions = df_all.last('Y')[[f'{item} (contribution, pp)' for item in demand_components]].\
transpose().sort_values(by=df_all.last('Y').index[0], ascending=False)
top_demand_contributions.index = [index.replace(' (contribution, pp)', '') for index in top_demand_contributions.index]

# Create dataframe for top three sources of growth on the production side
production_components = ['Agriculture', 'Industry (including construction)', 'Services']
top_production_contributions = df_all.last('Y')[[f'{item} (contribution, pp)' for item in production_components]].\
transpose().sort_values(by=df_all.last('Y').index[0], ascending=False)
top_production_contributions.index = [index.replace(' (contribution, pp)', '') for index in top_production_contributions.index]

In [181]:
# Create blank .docx where the output will be exported
doc = docx.Document()

# Add title (heading level 0)
doc.add_heading(f'{country_codes.country_name[country]}: Asian Development Outlook', level=0)

# Add a heading for GDP growth section
doc.add_heading(f'GDP growth in {dt.date.today().year - 1}', level=1)

# Set placeholders
change = ''
if df_all['GDP growth (%)'].last('2Y').values[-1] > df_all['GDP growth (%)'].last('2Y').values[-2]:
    change = random.choice(['increased', 'picked up', 'grew'])
elif df_all['GDP growth (%)'].last('2Y').values[-1] == df_all['GDP growth (%)'].last('2Y').values[-2]:
    change = random.choice(['remained at', 'stayed at'])
else:
    change = random.choice(['decreased', 'contracted', 'slowed down'])

# Add a paragraph on GDP growth comparison
p = doc.add_paragraph(f"{'Overall GDP growth'}: GDP growth {change} from \
{df_all['GDP growth (%)'].last('2Y').values[-2]}% in \
{df_all['GDP growth (%)'].last('2Y').index[-2].year} to \
{df_all['GDP growth (%)'].last('2Y').values[-1]}% in \
{df_all['GDP growth (%)'].last('2Y').index[-1].year}.", style='List Number')
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

# Add a paragraph on the top three components with biggest contributions
p = doc.add_paragraph(f"Main sources of growth: On the demand side, \
{top_demand_contributions.index[0].lower()} \
contributed the most with {str(top_demand_contributions.values[0]).replace('[', '').replace(']', '')} \
percentage points (pp), followed by {top_demand_contributions.index[1].lower()} \
({str(top_demand_contributions.values[1]).replace('[', '').replace(']', '')}pp) and \
{top_demand_contributions.index[2].lower()} \
({str(top_demand_contributions.values[2]).replace('[', '').replace(']', '')}pp). \
On the production side, {top_production_contributions.index[0].lower()} and \
{top_production_contributions.index[1].lower()} contributed \
{str(top_production_contributions.values[0]).replace('[', '').replace(']', '')}pp and \
{str(top_demand_contributions.values[1]).replace('[', '').replace(']', '')}pp to overall growth, \
respectively.", 
                      style='List Number')
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

# Add a paragraph on the component with the least share
p = doc.add_paragraph(f"Other sources of growth: On the other hand, \
{top_demand_contributions.index[3].lower()} had a \
{'positive' if top_demand_contributions.values[3] > 0 else 'negative'} contribution to growth at \
{str(abs(top_demand_contributions.values[3])).replace('[', '').replace(']', '')}pp. \
Meanwhile, {top_production_contributions.index[2].lower()} contributed \
{str(abs(top_production_contributions.values[2])).replace('[', '').replace(']', '')}pp.", 
                      style='List Number')
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [182]:
# Create the dataframe that will be plotted
df_chart_gdp = df_all[['GDP growth (%)'] + [f'{item} (contribution, pp)' \
                                            for item in demand_components + production_components]].dropna()
df_chart_gdp.index = df_chart_gdp.index.year
df_chart_gdp.reset_index(inplace=True)

# Create a Pandas Excel writer using xlsxwriter as the engine
writer = pd.ExcelWriter(f'{country}-charts.xlsx', engine='xlsxwriter')
df_chart_gdp[['date', 'GDP growth (%)'] + [f'{item} (contribution, pp)' for item in demand_components]].\
to_excel(writer, sheet_name='GDP-demand', index=False)
df_chart_gdp[['date', 'GDP growth (%)'] + [f'{item} (contribution, pp)' for item in production_components]].\
to_excel(writer, sheet_name='GDP-production', index=False)

# Create a blank canvas
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
plt.style.use('default')

# Create a stacked bar chart of the components
df_chart_gdp[[f'{item} (contribution, pp)' for item in demand_components]].plot(kind='bar', 
                                                                                stacked=True, ax=axs[0])

df_chart_gdp[[f'{item} (contribution, pp)' for item in production_components]].plot(kind='bar', 
                                                                                stacked=True, ax=axs[1])

# Create a line plot of the GDP growth series
for i in range(len(axs)):
    axs[i].plot(df_chart_gdp.loc[:, 'GDP growth (%)'], lw=2.5, marker='D', markersize=10, 
        color='black')
    # Set labels in the x-axis
    axs[i].set_xticklabels(df_chart_gdp.date, rotation=0)
    axs[i].set_ylabel('percentage points')
    axs[i].yaxis.set_major_locator(ticker.MaxNLocator(6))
    axs[i].set_xlabel('')

# Set legend
axs[0].legend(['GDP growth (%)'] + demand_components)
axs[1].legend(['GDP growth (%)'] + production_components)

# Set graph formatting and save to local folder
axs[0].set_title('Demand-side contributions to growth')
axs[1].set_title('Supply-side contributions to growth')

plt.tight_layout()
plt.savefig(f'{country}_GDP_components.png')
plt.close()

# Add picture to the document
graph = doc.add_picture(f'{country}_GDP_components.png', width=Cm(14), height=Cm(7))
last_paragraph = doc.paragraphs[-1]
last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

In [183]:
# Add a heading for Expenditure Approach section
doc.add_heading('Expenditure approach', level=1)

# Write paragraphs for each demand component using a loop
for item in demand_components:
    change = ''
    if df_all[f'{item} (annual growth, %)'].last('2Y').values[-1] > df_all[f'{item} (annual growth, %)'].last('2Y').values[-2]:
        change = random.choice(['increased', 'picked up', 'grew'])
    elif df_all[f'{item} (annual growth, %)'].last('2Y').values[-1] == df_all[f'{item} (annual growth, %)'].last('2Y').values[-2]:
        change = random.choice(['remained at', 'stayed at'])
    else:
        change = random.choice(['decreased', 'contracted', 'slowed down'])
    # Add a paragraph on GDP growth comparison
    p = doc.add_paragraph(f"{item}: {item} growth {change} from \
{df_all[f'{item} (annual growth, %)'].last('2Y').values[-2]}% in \
{df_all[f'{item} (annual growth, %)'].last('2Y').index[-2].year} to \
{df_all[f'{item} (annual growth, %)'].last('2Y').values[-1]}% in \
{df_all[f'{item} (annual growth, %)'].last('2Y').index[-1].year}, accounting for \
{df_all[f'{item} (% of GDP)'].last('2Y').values[-1]}% of GDP.", style='List Number')
    p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    
# Add a heading for Production Approach section
doc.add_heading('Production approach', level=1)

# Write paragraphs for each demand component using a loop
for item in production_components:
    change = ''
    if df_all[f'{item} (annual growth, %)'].last('2Y').values[-1] > df_all[f'{item} (annual growth, %)'].last('2Y').values[-2]:
        change = random.choice(['increased', 'picked up', 'grew'])
    elif df_all[f'{item} (annual growth, %)'].last('2Y').values[-1] == df_all[f'{item} (annual growth, %)'].last('2Y').values[-2]:
        change = random.choice(['remained at', 'stayed at'])
    else:
        change = random.choice(['decreased', 'contracted', 'slowed down'])
    # Add a paragraph on GDP growth comparison
    p = doc.add_paragraph(f"{item}: {item} growth {change} from \
{df_all[f'{item} (annual growth, %)'].last('2Y').values[-2]}% in \
{df_all[f'{item} (annual growth, %)'].last('2Y').index[-2].year} to \
{df_all[f'{item} (annual growth, %)'].last('2Y').values[-1]}% in \
{df_all[f'{item} (annual growth, %)'].last('2Y').index[-1].year}, accounting for \
{df_all[f'{item} (% of GDP)'].last('2Y').values[-1]}% of GDP.", style='List Number')
    p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [184]:
# Write section on income approach

# Add a heading for the income approach section
doc.add_heading(f'Income approach', level=1)

# Set placeholders
change = ''
if df_all['Unemployment rate (%)'].last('2Y').values[-1] > df_all['Unemployment rate (%)'].last('2Y').values[-2]:
    change = random.choice(['worsened', 'increased', 'jumped'])
elif df_all['GDP growth (%)'].last('2Y').values[-1] == df_all['GDP growth (%)'].last('2Y').values[-2]:
    change = random.choice(['remained at', 'stayed at'])
else:
    change = random.choice(['improved', 'declined', 'slowed down'])

# Add a paragraph on unemployment rate
p = doc.add_paragraph(f"{'Employment'}: Unemployment rate {change} from \
{df_all['Unemployment rate (%)'].last('2Y').values[-2]}% in \
{df_all['Unemployment rate (%)'].last('2Y').index[-2].year} to \
{df_all['Unemployment rate (%)'].last('2Y').values[-1]}% in \
{df_all['Unemployment rate (%)'].last('2Y').index[-1].year}.", style='List Number')
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [185]:
# Add a heading for the income approach section
doc.add_heading(f'Inflation', level=1)

# Set placeholders
change = ''
if df_all['Inflation rate (%)'].last('2Y').values[-1] > df_all['Inflation rate (%)'].last('2Y').values[-2]:
    change = random.choice(['worsened', 'increased', 'jumped'])
elif df_all['Inflation rate (%)'].last('2Y').values[-1] == df_all['Inflation rate (%)'].last('2Y').values[-2]:
    change = random.choice(['remained at', 'stayed at'])
else:
    change = random.choice(['improved', 'declined', 'slowed down'])

# Add a paragraph on unemployment rate
p = doc.add_paragraph(f"{'Inflation'}: Inflation rate {change} from \
{df_all['Inflation rate (%)'].last('2Y').values[-2]}% in \
{df_all['Inflation rate (%)'].last('2Y').index[-2].year} to \
{df_all['Inflation rate (%)'].last('2Y').values[-1]}% in \
{df_all['Inflation rate (%)'].last('2Y').index[-1].year}.", style='List Number')
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [186]:
# Create the dataframe that will be plotted
df_chart_inf_cab = pd.DataFrame(df_all[['Inflation rate (%)', 'Current account balance (% of GDP)']].\
                                last('5Y'))
df_chart_inf_cab.index = df_chart_inf_cab.index.year
df_chart_inf_cab.reset_index(inplace=True)

# Create a Pandas Excel writer using xlsxwriter as the engine
inf_cab = ['Inflation rate (%)', 'Current account balance (% of GDP)']
for item in inf_cab:
    df_chart_inf_cab[['date', f'{item}']].to_excel(writer, sheet_name=f'{item[:31]}', index=False)

# Close the Pandas Excel writer and output the Excel file
writer.save()

# Create a blank canvas
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
plt.style.use('default')

# Create a bar chart of inflation and CAB
df_chart_inf_cab['Inflation rate (%)'].plot(kind='bar', ax=axs[0])
df_chart_inf_cab['Current account balance (% of GDP)'].plot(kind='bar', ax=axs[1])

# Set graph formatting
for i in range(len(axs)):
    axs[i].set_xticklabels(df_chart_inf_cab.date, rotation=0)
    axs[i].set_xlabel('')
    axs[i].axes.get_yaxis().set_visible(False)

axs[0].set_title('Inflation rate (%)')
axs[1].set_title('Current account balance (% of GDP)')

# Set data labels for inflation
labels_inf = df_chart_inf_cab['Inflation rate (%)']
rects_inf = axs[0].patches

for rect, label in zip(rects_inf, labels_inf):
    height = rect.get_height()
    axs[0].text(rect.get_x() + rect.get_width() / 2, height, str(label) + '%',
                ha='center', va='bottom')
    
# Set data labels for CAB
labels_cab = df_chart_inf_cab['Current account balance (% of GDP)']
rects_cab = axs[1].patches

for rect, label in zip(rects_cab, labels_cab):
    height = rect.get_height()
    axs[1].text(rect.get_x() + rect.get_width() / 2, height, str(label) + '%',
                ha='center', va='bottom')

# Save to local folder
plt.savefig(f'{country}_inflation_cab.png')
plt.close()

# Add picture to the document
graph = doc.add_picture(f'{country}_inflation_cab.png', width=Cm(14), height=Cm(7))
last_paragraph = doc.paragraphs[-1]
last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

In [187]:
# Add a heading for the CAB analysis section
doc.add_heading(f'Current Account Balance', level=1)

# Set placeholders
sign_latest = 'surplus' if df_all['Current account balance (% of GDP)'].last('2Y').values[-1] > 0 else 'deficit'
sign_old = 'surplus' if df_all['Current account balance (% of GDP)'].last('2Y').values[-2] > 0 else 'deficit'

# Add a paragraph on unemployment rate
p = doc.add_paragraph(f"Current account balance (CAB): CAB recorded a {sign_latest} at \
{abs(df_all['Current account balance (% of GDP)'].last('2Y').values[-1])}% of GDP in \
{df_all['Current account balance (% of GDP)'].last('2Y').index[-1].year}. \
Net trade in goods and services reached \
USD {round(df_all['Net trade in goods and services (current USD)'].last('2Y').values[-1] / 1e9, 1)} \
billion. In {df_all['Current account balance (% of GDP)'].last('2Y').index[-2].year}, CAB posted a \
{sign_latest} at {abs(df_all['Current account balance (% of GDP)'].last('2Y').values[-2])}% of GDP.", 
                      style='List Number')
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

uitoken = '1deb01c07ca0da84f8c0cb7a9f2ecbbb'

url_destination = f"https://comtrade.un.org/api/get/plus?max=100000&type=C&freq=A&px=HS&ps={dt.date.today().year - 1}&r={country_codes.comtrade_code[country]}&p=all&rg=2&cc=TOTAL&uitoken={uitoken}&fmt=csv"
df_destination = pd.read_csv(url_destination)
total_exports = df_destination[df_destination['Partner']=='World'].loc[:, 'Trade Value (US$)'].values[0]
df_destination = df_destination.assign(share=lambda x: (x['Trade Value (US$)'] / total_exports * 100).round(1))
top_destination = df_destination.sort_values(by='share', ascending=False)[['Partner', 'share']].reset_index(drop=True).iloc[1:6, :]

url_origin = f"https://comtrade.un.org/api/get/plus?max=100000&type=C&freq=A&px=HS&ps={dt.date.today().year - 1}&r={country_codes.comtrade_code[country]}&p=all&rg=1&cc=TOTAL&uitoken={uitoken}&fmt=csv"
df_origin = pd.read_csv(url_origin)
total_imports = df_origin[df_origin['Partner']=='World'].loc[:, 'Trade Value (US$)'].values[0]
df_origin = df_origin.assign(share=lambda x: (x['Trade Value (US$)'] / total_imports * 100).round(1))
top_origin = df_origin.sort_values(by='share', ascending=False)[['Partner', 'share']].reset_index(drop=True).iloc[1:6, :]

url_exports = f"https://comtrade.un.org/api/get/plus?max=100000&type=C&freq=A&px=HS&ps={dt.date.today().year - 1}&r={country_codes.comtrade_code[country]}&p=0&rg=2&cc=AG2&uitoken={uitoken}&fmt=csv"
df_exports = pd.read_csv(url_exports)
df_exports = df_exports.assign(share=lambda x: (x['Trade Value (US$)'] / total_exports * 100).round(1))
top_exports = df_exports.iloc[1:, :].sort_values(by='share', ascending=False)[['Commodity', 'share']].head(5).reset_index(drop=True)

url_imports = f"https://comtrade.un.org/api/get/plus?max=100000&type=C&freq=A&px=HS&ps={dt.date.today().year - 1}&r={country_codes.comtrade_code[country]}&p=0&rg=1&cc=AG2&uitoken={uitoken}&fmt=csv"
df_imports = pd.read_csv(url_imports)
df_imports = df_imports.assign(share=lambda x: (x['Trade Value (US$)'] / total_imports * 100).round(1))
top_imports = df_imports.iloc[1:, :].sort_values(by='share', ascending=False)[['Commodity', 'share']].head(5).reset_index(drop=True)

p = doc.add_paragraph(f"Top exports partners: Top exports destinations in {dt.date.today().year - 1} are \
{top_destination['Partner'].values[0]} ({top_destination['share'].values[0]}% of total exports), \
{top_destination['Partner'].values[1]} ({top_destination['share'].values[1]}%), \
{top_destination['Partner'].values[2]} ({top_destination['share'].values[2]}%), \
{top_destination['Partner'].values[3]} ({top_destination['share'].values[3]}%),  and \
{top_destination['Partner'].values[4]} ({top_destination['share'].values[4]}%).",
                      style='List Number')
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

p = doc.add_paragraph(f"Top export commodities: Top export commodities include \
{top_exports['Commodity'].values[0].lower()} ({top_exports['share'].values[0]}% of total exports), \
{top_exports['Commodity'].values[1].lower()} ({top_exports['share'].values[1]}%), \
{top_exports['Commodity'].values[2].lower()} ({top_exports['share'].values[2]}%), \
{top_exports['Commodity'].values[3].lower()} ({top_exports['share'].values[3]}%),  and \
{top_exports['Commodity'].values[4].lower()} ({top_exports['share'].values[4]}%).",
                      style='List Number')
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

p = doc.add_paragraph(f"Top imports partners: Top imports origins in {dt.date.today().year - 1} are \
{top_origin['Partner'].values[0]} ({top_origin['share'].values[0]}% of total imports), \
{top_origin['Partner'].values[1]} ({top_origin['share'].values[1]}%), \
{top_origin['Partner'].values[2]} ({top_origin['share'].values[2]}%), \
{top_origin['Partner'].values[3]} ({top_origin['share'].values[3]}%),  and \
{top_origin['Partner'].values[4]} ({top_origin['share'].values[4]}%).",
                      style='List Number')
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

p = doc.add_paragraph(f"Top import commodities include \
{top_imports['Commodity'].values[0].lower()} ({top_imports['share'].values[0]}% of total imports), \
{top_imports['Commodity'].values[1].lower()} ({top_imports['share'].values[1]}%), \
{top_imports['Commodity'].values[2].lower()} ({top_imports['share'].values[2]}%), \
{top_imports['Commodity'].values[3].lower()} ({top_imports['share'].values[3]}%),  and \
{top_imports['Commodity'].values[4].lower()} ({top_imports['share'].values[4]}%).",
                      style='List Number')
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [188]:
# Create a list of CECL country codes
adb_country_codes = ['AFG', 'ARM', 'AZE', 'BAN', 'BHU', 'BRU', 'CAM', 'PRC', 'FIJ', 'HKG', 'GEO', 'IND',
                     'INO', 'KAZ', 'KIR', 'KGZ', 'KOR', 'LAO', 'MLD', 'RMI', 'FSM', 'MON', 'MYA', 'NEP', 
                     'PAK', 'PAL', 'PNG', 'PHL', 'SAM', 'SOL', 'SRI', 'TAJ', 'TAP', 'THA', 'TIM', 'TON', 
                     'TKM', 'TUV', 'UZB', 'VAN', 'VIE', 'RUS', 'MAL', 'SIN', 'COO', 'RMI', 'NAU', 'NIU']

# Create a list of Haver country codes
haver_country_codes = ['512', '911', '912', '513', '514', '516', '522', '924', '819', '532', '915', 
                       '534', '536', '916', '826', '917', '542', '544', '556', '867', '868', '948', 
                       '518', '558', '564', '565', '853', '566', '862', '813', '524', '923', '528',
                       '578', '537', '866', '925', '869', '927', '846', '582', '922', '548', '576', 
                       np.nan, '867', np.nan, np.nan]

haver_codes = pd.DataFrame(haver_country_codes, index=adb_country_codes, columns=['haver_code'])
country_codes = country_codes.join(haver_codes)

In [189]:
import Haver
import calendar

consumer_codes = ['VCC', 'VCE']
consumer_names = ['Consumer confidence index (NSA, % balance)', 
                  'Consumer expectations index (NSA, % balance)']
consumer_country = [country_codes.haver_code[country] + item for item in consumer_codes]
consumer_indicators = ['N' + item for item in consumer_country]
df_consumer = Haver.data(consumer_indicators, database='EMERGE', dates=True)
df_consumer.columns = consumer_names

production_codes =  ['D', 'DM', 'DN', 'DVW', 'DVU']
production_names = ['Industrial production (index)', 'Manufacturing (index)',
                    'Mining and quarrying (index)', 
                    'Water supply, sewerage, waste management & remediation (index)', 
                    'Electricity, gas, steam and air conditioning supply (index)']
production_country = [country_codes.haver_code[country] + item for item in production_codes]
production_indicators = ['S' + item for item in production_country]
df_production = Haver.data(production_indicators, database='EMERGE', dates=True)
df_production.columns = production_names
df_production = (df_production.pct_change(12, fill_method=None) * 100).round(1).tail(12).dropna()

cpi_codes = ['PC', 'PCF', 'PCH', 'PCT', 'PCC', 'PCM', 'PCR', 'PCE']
cpi_names = ['All Items', 'Food', 'Housing, rent, water, electricity, gas & other fuels',
             'Transportation', 'Communication', 'Health/medical care', 'Recreation', 'Education']
cpi_country = [country_codes.haver_code[country] + item for item in cpi_codes]
cpi_indicators = ['H' + item for item in cpi_country]
df_cpi = Haver.data(cpi_indicators, database='EMERGE', dates=True)
df_cpi.columns = cpi_names
df_cpi = (df_cpi.pct_change(12, fill_method=None) * 100).round(1).tail(12).dropna()

policyrate_codes = ['RTAR']
policyrate_names = ['Policy rate (EOP, %)']
policyrate_country = [country_codes.haver_code[country] + item for item in policyrate_codes]
policyrate_indicators = ['N' + item for item in policyrate_country]
df_policyrate = Haver.data(policyrate_indicators, database='EMERGE', dates=True)
df_policyrate.columns = policyrate_names

In [190]:
# Write the section on Outlook

# Add a heading for the Outlook section
doc.add_heading('Outlook', level=1)

# Set placeholders
change_confidence = ''
if df_consumer[f'Consumer confidence index (NSA, % balance)'].last('5Q').values[-1] > \
df_consumer[f'Consumer confidence index (NSA, % balance)'].last('5Q').values[-5]:
    change_confidence = random.choice(['improved', 'increased', 'jumped'])
elif df_consumer[f'Consumer confidence index (NSA, % balance)'].last('5Q').values[-1] == \
df_consumer[f'Consumer confidence index (NSA, % balance)'].last('5Q').values[-5]:
    change_confidence = random.choice(['remained at', 'stayed at'])
else:
    change_confidence = random.choice(['worsened', 'declined', 'contracted'])
    
# Set placeholders
change_expectations = ''
if df_consumer[f'Consumer expectations index (NSA, % balance)'].last('5Q').values[-1] > \
df_consumer[f'Consumer expectations index (NSA, % balance)'].last('5Q').values[-5]:
    change_expectations = random.choice(['improved', 'increased', 'jumped'])
elif df_consumer[f'Consumer expectations index (NSA, % balance)'].last('5Q').values[-1] == \
df_consumer[f'Consumer expectations index (NSA, % balance)'].last('5Q').values[-5]:
    change_expectations = random.choice(['remained at', 'stayed at'])
else:
    change_expectations = random.choice(['worsened', 'declined', 'contracted'])
    
quarter = ''
if df_consumer.last('5Q').index[-1].strftime('%Y-%m-%d')[5:7] == '03':
    quarter = 'Q1'
elif df_consumer.last('5Q').index[-1].strftime('%Y-%m-%d')[5:7] == '06':
    quarter = 'Q2'
elif df_consumer.last('5Q').index[-1].strftime('%Y-%m-%d')[5:7] == '09':
    quarter = 'Q3'
else:
    quarter = 'Q4'
    
# Add a paragraph on Confidence index
p = doc.add_paragraph(f"Consumer confidence: Consumer confidence index {change_confidence} to \
{df_consumer[f'Consumer confidence index (NSA, % balance)'].last('5Q').values[-1]} points in \
{quarter} of {df_consumer.last('5Q').index[-1].strftime('%Y-%m-%d')[0:4]} from \
{df_consumer[f'Consumer confidence index (NSA, % balance)'].last('5Q').values[-5]} points \
in the same quarter in the previous year, reflecting more \
{'pessimistic' if df_consumer[f'Consumer confidence index (NSA, % balance)'].last('5Q').values[-1] < 0 else 'optimistic'} \
consumer sentiments. \
Consumer expectations about the general economic situation in the \
next 12 months {change_expectations} to \
{df_consumer[f'Consumer expectations index (NSA, % balance)'].last('5Q').values[-1]} points from \
{df_consumer[f'Consumer expectations index (NSA, % balance)'].last('5Q').values[-5]} points, \
indicating {'negative' if df_consumer[f'Consumer expectations index (NSA, % balance)'].last('5Q').values[-1] < 0 else 'positive'} \
expections about the country's current and future economic situation.", 
                      style='List Number')
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

# Set placeholders
change_production = {}
for item in production_names:
    if df_production[f'{item}'].values[-1] > df_production[f'{item}'].values[-2]:
        change_production[f'{item}'] = random.choice(['improved', 'increased', 'jumped'])
    elif df_production[f'{item}'].values[-1] == df_production[f'{item}'].values[-2]:
        change_production[f'{item}'] = random.choice(['remained at', 'stayed at'])
    else:
        change_production[f'{item}'] = random.choice(['worsened', 'declined', 'contracted'])

# Add a paragraph on Production indices
p = doc.add_paragraph(f"Industrial production: Indutrial output \
{change_production['Industrial production (index)']} to \
{df_production['Industrial production (index)'].values[-1]}% year-on-year in \
{calendar.month_name[int(df_production.index[-1].strftime('%Y-%m-%d')[5:7])]} this year \
from {df_production['Industrial production (index)'].values[-2]}% in the \
previous month. Looking at the details, manufacturing \
{change_production['Manufacturing (index)']} to \
{df_production['Manufacturing (index)'].values[-1]}% from \
{df_production['Manufacturing (index)'].values[-2]}%, while \
mining and quarrying {change_production['Mining and quarrying (index)']} to \
{df_production['Mining and quarrying (index)'].values[-1]}% from \
{df_production['Mining and quarrying (index)'].values[-2]}%. \
Meanwhile, water supply, sewerage, waste management & remediation \
{change_production['Water supply, sewerage, waste management & remediation (index)']} to \
{df_production['Water supply, sewerage, waste management & remediation (index)'].values[-1]}% from \
{df_production['Water supply, sewerage, waste management & remediation (index)'].values[-2]}%, while \
electricity, gas, steam and air conditioning supply \
{change_production['Electricity, gas, steam and air conditioning supply (index)']} to \
{df_production['Electricity, gas, steam and air conditioning supply (index)'].values[-1]}% from \
{df_production['Electricity, gas, steam and air conditioning supply (index)'].values[-2]}%.",
                      style='List Number')
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

# Set placeholders
change_cpi = {}
for item in cpi_names:
    if df_cpi[f'{item}'].values[-1] > df_cpi[f'{item}'].values[-2]:
        change_cpi[f'{item}'] = random.choice(['worsened', 'rose', 'jumped'])
    elif df_cpi[f'{item}'].values[-1] == df_cpi[f'{item}'].values[-2]:
        change_cpi[f'{item}'] = random.choice(['remained at', 'stayed at'])
    else:
        change_cpi[f'{item}'] = random.choice(['improved', 'declined', 'slowed down'])
        
# Add a paragraph on Production indices
p = doc.add_paragraph(f"Inflation: Inflation {change_cpi['All Items']} to \
{df_cpi['All Items'].values[-1]}% year-on-year in \
{calendar.month_name[int(df_cpi.index[-1].strftime('%Y-%m-%d')[5:7])]} this year \
from {df_cpi['All Items'].values[-2]}% in the \
previous month. Prices for food products {change_cpi['Food']} to \
{df_cpi['Food'].values[-1]}% from {df_cpi['Food'].values[-2]}%, while housing, rent, water, \
electricity, gas & other fuels {change_cpi['Housing, rent, water, electricity, gas & other fuels']} to \
{df_cpi['Housing, rent, water, electricity, gas & other fuels'].values[-1]}% from \
{df_cpi['Housing, rent, water, electricity, gas & other fuels'].values[-2]}%. \
Transportation {change_cpi['Transportation']} to \
{df_cpi['Transportation'].values[-1]}% from {df_cpi['Transportation'].values[-2]}%, while \
communication {change_cpi['Communication']} to \
{df_cpi['Communication'].values[-1]}% from {df_cpi['Communication'].values[-2]}%. Meanwhile, \
prices for health/medical care {change_cpi['Health/medical care']} to \
{df_cpi['Health/medical care'].values[-1]}% from {df_cpi['Health/medical care'].values[-2]}%, \
recreation {change_cpi['Recreation']} to \
{df_cpi['Recreation'].values[-1]}% from {df_cpi['Recreation'].values[-2]}%, and \
education {change_cpi['Education']} to \
{df_cpi['Education'].values[-1]}% from {df_cpi['Education'].values[-2]}%",
                      style='List Number')
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

# Set placeholders
change_policyrate = ''
if df_policyrate['Policy rate (EOP, %)'].values[-1] > df_policyrate['Policy rate (EOP, %)'].values[-2]:
    change_policyrate = random.choice(['tightened', 'increased', 'raised'])
elif df_policyrate['Policy rate (EOP, %)'].values[-1] == df_policyrate['Policy rate (EOP, %)'].values[-2]:
    change_policyrate = random.choice(['kept', 'maintained'])
else:
    change_expectations = random.choice(['eased', 'decreased', 'lowered'])

# Add a paragraph on Monetary policy
p = doc.add_paragraph(f"Monetary policy: {country_codes.country_name[country]}'s central bank \
{change_policyrate} the key interest rate at {df_policyrate['Policy rate (EOP, %)'].values[-1]}% in \
{calendar.month_name[int(df_policyrate.index[-1].strftime('%Y-%m-%d')[5:7])]} from \
{df_policyrate['Policy rate (EOP, %)'].values[-2]}% in \
{calendar.month_name[int(df_policyrate.index[-1].strftime('%Y-%m-%d')[5:7]) - 1]}.", 
                      style='List Number')
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

# # Add a paragraph on Fiscal policy
# p = doc.add_paragraph('Fiscal policy', style='List Number')

# Add a paragraph on Current events
p = doc.add_paragraph('Comments on current events', style='List Number')

# # Add a paragraph on leading indicators
# p = doc.add_paragraph('Leading indicators', style='List Number')

df_forecasts = pd.read_csv('consensus_forecasts_13nov.csv', index_col='country')
df_forecasts = df_forecasts.loc[df_forecasts.index == country].round(1)

# Add a paragraph on Consensus Forecasts
p = doc.add_paragraph(f"Outlook: On {df_forecasts.date.values[0]}, Consensus Economics panelists \
forecast {country_codes.country_name[country]}' economic growth ending in \
{df_forecasts.date.values[0][-4:]} at {df_forecasts['gdp_forecast_2020'].values[0]}%. In 2021, \
the panelists project growth to end the year at {df_forecasts['gdp_forecast_2021'].values[0]}%.", 
                      style='List Number')
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

# Add a paragraph on Risks
p = doc.add_paragraph('Risks', style='List Number')

# Save the .docx file into the local folder
doc.save(f'{country} ADO Quick Report.docx')

In [191]:
df_forecasts['gdp_forecast_2020'].values[0]

-4.0